In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# загружаем датасеты
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# убираем нунужные данные (остаются: возраст, пол, семья, класс каюты, должность в команде)
data = data.drop(columns=["Name", "Ticket", "PassengerId", "Fare", "Cabin", "Embarked"])
test = test.drop(columns=["Name", "Ticket", "PassengerId", "Fare", "Cabin", "Embarked"])

# Объединяем родственников в семью 
data["FamCnt"] = data["SibSp"] + data["Parch"]
data = data.drop(columns=["SibSp", "Parch"])
test["FamCnt"] = test["SibSp"] + test["Parch"]
test = test.drop(columns=["SibSp", "Parch"])

In [2]:
# заменяем пол на значения номер принадлежности: {0,1}
# класс и должнось разделяем на подклассы со свойством принадлежности: {0,1}
data["Sex"] = LabelEncoder().fit_transform(data["Sex"])
# data = pd.get_dummies(data, columns=["Pclass"], dtype=int)
test["Sex"] = LabelEncoder().fit_transform(test["Sex"])
# test = pd.get_dummies(test, columns=["Pclass"], dtype=int)

0.6161616161616161


C:\Users\stden\anaconda3\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but OneVsOneClassifier was fitted without feature names
  warnings.warn(


In [ ]:

# Заполняем пропущенные значения возрастов медианными значениями и переводим в целочисленный тип
data = data.fillna(data.median())
data["Age"] = data["Age"].astype(int)
test = test.fillna(test.mean())
test["Age"] = test["Age"].astype(int)

X = data.drop(columns=["Survived"])
Y = data["Survived"]

# создаем тренировочную и тестовую выборку
X_tr, X_t, Y_tr, Y_t = train_test_split(X, Y, test_size=0.33,random_state=42)

# print(X_tr, X_t, Y_tr, Y_t, sep='\n\n')

# Feature Scaling using StandardScaler
scaler = StandardScaler().fit(X_tr)
X_tr_std = scaler.transform(X_tr)
X_t_std = scaler.transform(X_t)
test_scaled = scaler.transform(test)

# метод опорных векторов с линейным ядром, классификатор 1 против 1
ovo = OneVsOneClassifier(SVC(kernel = 'linear')).fit(X_tr_std , Y_tr)
# предсказание
predictions = ovo.predict(test_scaled)

X_test = data.drop(columns=["Survived"])
Y_test = data["Survived"]
print(ovo.score(X_test, Y_test))


# создавем scv-файл
data = pd.read_csv("test.csv")
result = pd.DataFrame({'PassengerId': data['PassengerId'], 'Survived': predictions})
result.to_csv("result.csv", header=True, index=False)